In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
import seaborn as sns
import statistics
from tqdm import tqdm

In [3]:
def log_beta(beta0, i, c=1):
    beta = np.log(np.exp(c*beta0)+i)/c
    return beta

In [11]:
bounds = Bounds([0], [np.inf])
def langevin(x0, d, func, grad, maxiter = 100, eta0 = 0.001, beta0 = 1, beta_schedule = log_beta, c=1):
    """
    This code implements Gradient Langevin Algorithm with exact linesearch on the step size eta

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        eta0 (float): initial value for eta
        beta0 (float): initial value for beta
        beta_schedule (Callable): annealing schedule for temperature
        c (float): constant in logarithmic annealing schedule

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    x_list[0,:] = x0
    f_list = np.zeros((maxiter,))
    f_list[0] = func(x0)
    for i in range(1,maxiter):
        epsilon = np.random.normal(0, 1, d)
        beta = beta_schedule(beta0, i, c=c)
        def objective_function(eta):
            return func(x_list[i-1,:]- eta*grad(x_list[i-1,:]) + np.sqrt(2*eta/beta)*epsilon)
        # perform exact linesearch
        result = minimize(objective_function, eta0, method = "SLSQP", bounds=bounds)
        eta = result.x
        x_list[i,:] = x_list[i-1,:] - eta*grad(x_list[i-1,:]) + np.sqrt(2*eta/beta)*epsilon
        f_list[i] = func(x_list[i,:])
    return f_list, x_list

In [13]:
def underdamped_langevin(x0, 
                         d, 
                         func, 
                         grad, 
                         maxiter = 100, 
                         eta0 = 0.001, 
                         beta0 = 1, 
                         beta_schedule = log_beta, 
                         gamma = 0.1, 
                         c=0.1):
    """
    This code implements Underdamped Langevin Algorithm with exact linesearch on the step size eta

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        eta0 (float): initial value for eta
        beta0 (float): initial value for beta
        beta_schedule (Callable): annealing schedule for temperature
        gamma (float): friction coefficient
        c (float): constant in logarithmic annealing schedule

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    x_list[0,:] = x0
    f_list = np.zeros((maxiter,))
    f_list[0] = func(x0)
    v_list = np.zeros((maxiter,d))
    v_list[0,:] = np.zeros((d,)) 

    for i in range(1,maxiter):
        epsilon = np.random.normal(0, 1, d)
        beta = beta_schedule(beta0, i, c)
        v_list[i,:] = v_list[i-1,:] - eta0*(gamma*v_list[i-1,:] + grad(x_list[i-1,:])) + np.sqrt(2*eta0*gamma/beta)*epsilon
        x_list[i,:] = x_list[i-1,:] + eta0*v_list[i-1,:]
        f_list[i] = func(x_list[i,:])
    return f_list, x_list

In [15]:
def random_skew_symmetric(n):
    A = np.random.randn(n, n)*0.1
    return A - A.T  # This ensures the matrix is skew-symmetric

In [17]:
bounds = Bounds([0], [np.inf])
def nonreversible_langevin(x0, d, func, grad, maxiter = 100, eta0 = 0.001, beta0 = 1, beta_schedule = log_beta, c = 1):
    """
    This code implements Non-reversible Langevin Algorithm with exact linesearch on the step size eta

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        eta0 (float): initial value for eta
        beta0 (float): initial value for beta
        beta_schedule (Callable): annealing schedule for temperature
        c (float): constant in logarithmic annealing schedule

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    x_list[0,:] = x0
    f_list = np.zeros((maxiter,))
    f_list[0] = func(x0)
    AJ = random_skew_symmetric(d) + np.eye(d)

    for i in range(1,maxiter):
        epsilon = np.random.normal(0, 1, d)
        beta = beta_schedule(beta0, i, c)
        def objective_function(eta):
            return func(x_list[i-1,:]- eta*grad(x_list[i-1,:]) + np.sqrt(2*eta/beta)*epsilon)
        # perform exact linesearch
        result = minimize(objective_function, eta0, method = "SLSQP", bounds=bounds)
        eta = result.x
        beta = beta_schedule(beta0, i, c)
        x_list[i,:] = x_list[i-1,:] - eta*AJ.dot(grad(x_list[i-1,:])) + np.sqrt(2*eta/beta)*epsilon
        f_list[i] = func(x_list[i,:])
    return f_list, x_list

In [19]:
def plot_and_compare(d, func, grad, maxiter=400, eta0_gld=1e-2, eta0_nld=1e-2, beta0=1, beta_schedule=step_beta, c=1, var=1, gamma=1):
    """
    This code implements Gradient Langevin, Underdamped Langevin and Non-reversible Langevin and:
    1) plots the semi-log plot of average convergence curve;
    2) plots the boxplot of the final value of both algorithms;
    3) computes Q1, median and Q3 of the final value of both algorithms;

    Args:
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        eta0_gld (float): initial value for eta used in Gradient Langevin
        eta0_nld (float): initial value for eta used in Non-reversible Langevin
        beta0 (float): initial value for beta
        beta_schedule (Callable): annealing schedule for temperature
        c (float): constant in logarithmic annealing schedule
        var (float): variance of the distribution where the initial point is sampled from
        gamma (float): friction coefficient   
    """
    gld_result = None
    uld_result = None
    nld_result = None
    x = np.zeros((100,2))
    gld_x = np.zeros((100,2))
    uld_x = np.zeros((100,2))
    nld_x = np.zeros((100,2))
    for i in tqdm(range(100)):
        x0 = np.random.normal(0,var, size=(2,))
        x[i,:] = x0
        f_list_gld, x_list_gld = langevin(x0, d, func, grad, maxiter=maxiter, eta0 = eta0_gld, beta0 = beta0, beta1=beta1, beta_schedule = beta_schedule, c=c)
        f_list_uld, x_list_uld = underdamped_langevin(x0, d, func, grad, maxiter=maxiter, eta0 = eta0_gld, beta0 = beta0, beta1=beta1, beta_schedule = beta_schedule, c=c, gamma=gamma)
        f_list_nld, x_list_nld = nonreversible_langevin(x0, d, func, grad, maxiter=maxiter, eta0 = eta0_gld, beta0 = beta0, beta1=beta1, beta_schedule = beta_schedule, c=c)
        gld_x[i,:] = x_list_gld[-1,:]
        uld_x[i,:] = x_list_uld[-1,:]
        nld_x[i,:] = x_list_nld[-1,:]
        f_list_gld = np.array(f_list_gld)
        f_list_uld = np.array(f_list_uld)
        f_list_nld = np.array(f_list_nld)
        f_list_gld = f_list_gld.reshape((maxiter,1))
        f_list_uld = f_list_uld.reshape((maxiter,1))
        f_list_nld = f_list_nld.reshape((maxiter,1))
        if i == 0:
            gld_result = f_list_gld
            uld_result = f_list_uld
            nld_result = f_list_nld
        else:
            gld_result = np.concatenate((gld_result, f_list_gld), axis = 1)
            uld_result = np.concatenate((uld_result, f_list_uld), axis = 1)
            nld_result = np.concatenate((nld_result, f_list_nld), axis = 1)

    # Extract final output
    gld_final = gld_result[-1,:]
    uld_final = uld_result[-1,:]
    nld_final = nld_result[-1,:]

    mean_gld = np.mean(gld_result, axis=1)
    std_dev_gld = np.std(gld_result, axis=1)
    mean_uld = np.mean(uld_result, axis=1)
    std_dev_uld = np.std(uld_result, axis=1)
    mean_nld = np.mean(nld_result, axis=1)
    std_dev_nld = np.std(nld_result, axis=1)

    plt.plot(np.arange(0,maxiter), np.log10(mean_gld), label = "Gradient Langevin")
    plt.plot(np.arange(0,maxiter), np.log10(mean_uld), label = "Underdamped Langevin")
    plt.plot(np.arange(0,maxiter), np.log10(mean_nld), label = "Non-reversible Langevin")
    # plt.ylim(0,0.001)
    plt.legend()
    plt.xlabel("Number of Gradient Evaluations")
    plt.ylabel("log(f(x_k) - f^*)")
    plt.savefig("mean_variant.jpg")

    plt.figure(figsize=(4,6))
    sns.boxplot(data=[gld_final, uld_final, nld_final], palette='pastel')
    plt.title("Boxplot of Final Objective Value")
    plt.ylabel("Function Value")
    plt.xticks([0,1,2], ["GLA", 'ULA', 'NLA'])  # if you just have one list
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)
    plt.savefig("boxplot_variant.jpg")
    plt.show()

    gld_med = statistics.median(gld_final)
    gld_mean = statistics.mean(gld_final)
    # quartiles – split into 4 bins
    q1, q2, q3 = statistics.quantiles(gld_final, n=4)  # q2 == median
    print("Q1, Q2(median), Q3, mean for Gradient Langevin:", q1, q2, q3, gld_mean)
    
    uld_med = statistics.median(uld_final)
    uld_mean = statistics.mean(uld_final)
    # quartiles – split into 4 bins
    q1, q2, q3 = statistics.quantiles(uld_final, n=4)  # q2 == median
    print("Q1, Q2(median), Q3, mean for Underdamped Langevin:", q1, q2, q3, uld_mean)

    nld_med = statistics.median(nld_final)
    nld_mean = statistics.mean(nld_final)
    # quartiles – split into 4 bins
    q1, q2, q3 = statistics.quantiles(nld_final, n=4)  # q2 == median
    print("Q1, Q2(median), Q3, mean for Underdamped Langevin:", q1, q2, q3, nld_mean)


NameError: name 'step_beta' is not defined